![RNN logic](../image/RNN1.png)

matmul
(A, B) @ (B, C) = (A, C)

+
(N, H) + (N, H) = (N, H)
if can use broadcast
(N, H) + (H,) = (N, H)



In [ ]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
    
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        
        #가중치 연산.
        #shape: (N, H) @ (H, H) + (N, D) @ (D, H) + (H,) = (N, H)
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        
        #tanh 활성함수
        #shape: (N, H)
        h_next = np.tanh(t)
        print
        
        self.cache = (x, h_prev, h_next)
        return h_next
    
    #dh_next Shape: (N, H)
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        
        #x Shape: (N, D)
        #h_prev Shape: (N, H)
        #h_next Shape: (N, H)
        x, h_prev, h_next = self.cache
        
        #tanh 활성함수의 미분
        #dt Shape: (N, H)
        dt = dh_next * (1 - h_next ** 2)
        
        #가중치 연산의 미분
        #db Shape: (H,)
        db = np.sum(dt, axis=0)
        
        #이전 은닉 상태의 미분
        #dWh Shape: (H, N) @ (N, H) = (H, H)
        dWh = np.matmul(h_prev.T, dt)
        
        #입력의 미분
        #dWx Shape: (D, N) @ (N, H) = (D, H)
        dWx = np.matmul(x.T, dt)
        
        #이전 은닉 상태의 미분
        #dh_prev Shape: (N, H) @ (H, H) = (N, H)
        dh_prev = np.matmul(dt, Wh.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dh_prev

N : Batch Size

T : Time Step Size

D : Input Dimension Size

H : Hidden Dimension Size

In [2]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        
        self.h, self.dh = None, None
        self.stateful = stateful
    
    def set_state(self, h):
        self.h = h
    
    def reset_state(self):
        self.h = None

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        
        self.h, self.dh = None, None
        self.stateful = stateful
    
    def set_state(self, h):
        self.h = h
    
    def reset_state(self):
        self.h = None
    
    #shape: (N, T, D)
    def forward(self, xs):
        #Wx shape: (D, H)
        #Wh shape: (H, H)
        #b shape: (H,)
        Wx, Wh, b = self.params
        
        # 입력 데이터 형상
        N, T, D = xs.shape
        
        # 은닉 상태 형상
        D, H = Wx.shape
        
        self.layers = []
        
        # 은닉 상태 초기화
        # 다음 Time RNN클래스에서 사용할 은닉 상태 변수
        #hs shape: (N, T, H)
        # N개의 데이터, T개의 시간 단계, H개의 은닉 상태
        hs = np.empty((N, T, H), dtype='f')
        
        
        
        # 다음 Time RNN클래스에서 사용한다면 stateful=True로 설정해야 함
        # 또는 초기 호출시 은닉 상태를 초기화해야 함
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
        
        # 시간 차원에 대한 반복
        # RNN1 RNN2 RNN3 순으로 처리
         
        
        # 은닉 상태 반환
        return hs
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        # 역전파 시작
        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        
        grads = [0,0,0]
        
        # 시간 차원에 대한 반복
        # RNN3 RNN2 RNN1 순으로 처리
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
            
            dh = dh_next
        
        for i, grad in enumerate(grads):
            self.grads[i] += grad
        
        self.dh = dh
        
        return dxs

In [4]:
import sys
sys.path.append('./deep/')
import numpy as np
from common.time_layers import *

class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V,D) / 100).astype('f')
        rnn_Wx  = (rn(D,H) / np.sqrt(D)).astype('f')
        rnn_Wh  = (rn(H,H) / np.sqrt(H)).astype('f')
        rnn_b   = np.zeros(H).astype('f')
        
        affine_W = (rn(H,V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]
        
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        self.rnn_layer.reset_state()
        

In [5]:
import sys

sys.path.append('./deep/')

import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import Adam
from dataset import ptb
# from ch05.simple_rnnlm import SimpleRnnlm

from common.util import clip_grads  # 추가

batch_size = 10
wordvec_size = 100
hidden_size = 100
time_size = 5
lr = 0.001
max_epoch = 1000

max_grad = 5.0  # Gradient clipping 추가

corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 10000

corpus = corpus[:corpus_size]

vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]
data_size = len(xs)

print('말뭉치 크기: %d, 어휘 수: %d' % (corpus_size, vocab_size))

max_iters = data_size // (batch_size * time_size)
total_loss = 0
loss_count = 0
ppl_list = []

model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = Adam(lr)

jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]

for epoch in range(max_epoch):
    time_idx = 0  # 에포크마다 리셋
    for iters in range(max_iters):
        # 2D 배열로 초기화
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1
        
        loss = model.forward(batch_x, batch_t)
        model.backward(dout=1)
        
        if max_grad is not None:
            clip_grads(model.grads, max_grad)
        
        
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1
    
    ppl = np.exp(total_loss / loss_count)
    # print 문 수정
    print('| 에포크 %d | 단어 수 %d | 손실 %f | perplexity %f' % (epoch + 1, data_size, total_loss / loss_count, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

말뭉치 크기: 10000, 어휘 수: 10000
| 에포크 1 | 단어 수 9999 | 손실 7.317571 | perplexity 1506.539646
| 에포크 2 | 단어 수 9999 | 손실 5.972488 | perplexity 392.480824
| 에포크 3 | 단어 수 9999 | 손실 5.573101 | perplexity 263.249077
| 에포크 4 | 단어 수 9999 | 손실 5.180556 | perplexity 177.781698
| 에포크 5 | 단어 수 9999 | 손실 4.790765 | perplexity 120.393400
| 에포크 6 | 단어 수 9999 | 손실 4.419150 | perplexity 83.025673
| 에포크 7 | 단어 수 9999 | 손실 4.071925 | perplexity 58.669807
| 에포크 8 | 단어 수 9999 | 손실 3.735256 | perplexity 41.898750
| 에포크 9 | 단어 수 9999 | 손실 3.402706 | perplexity 30.045289
| 에포크 10 | 단어 수 9999 | 손실 3.089447 | perplexity 21.964938
| 에포크 11 | 단어 수 9999 | 손실 2.794650 | perplexity 16.356896
| 에포크 12 | 단어 수 9999 | 손실 2.523296 | perplexity 12.469630
| 에포크 13 | 단어 수 9999 | 손실 2.274477 | perplexity 9.722831
| 에포크 14 | 단어 수 9999 | 손실 2.045598 | perplexity 7.733781
| 에포크 15 | 단어 수 9999 | 손실 1.839805 | perplexity 6.295310
| 에포크 16 | 단어 수 9999 | 손실 1.660145 | perplexity 5.260072
| 에포크 17 | 단어 수 9999 | 손실 1.499509 | perplexity 4.47